<a href="https://colab.research.google.com/github/AUT-Student/CN-HW1/blob/main/ComplexNetwork_HW1_Q4_Q5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graph

In [1]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Edge():
  """ This class stores data of an directed edge
  
  Args:
    source: The source of an edge
    destination: The destination of an edge

  """
  source: int
  destination: int

In [2]:
class Graph():
  """ This class stores data of a directed graph. Furtermore some functions defines here"""
  def __init__(self):
    self.nodes = set()
    self.edges = set()
    self.neighbour_set = dict()
    self.neighbour_reverse_set = dict()

  def add_node(self, node):
    self.nodes.add(node)

    if node not in self.neighbour_set:
      self.neighbour_set[node] = set()

    if node not in self.neighbour_reverse_set:
      self.neighbour_reverse_set[node] = set()

  def add_edge(self, edge:Edge):
    self.add_node(edge.source)
    self.add_node(edge.destination)

    self.neighbour_set[edge.source].add(edge.destination)
    self.neighbour_reverse_set[edge.destination].add(edge.source)
    self.edges.add(edge)

  def remove_edge(self, edge:Edge):
    self.neighbour_set[edge.source].remove(edge.destination)
    self.neighbour_reverse_set[edge.destination].remove(edge.source)
    self.edges.remove(edge)

  def is_exist(self, edge:Edge):
    return edge in self.edges

  def get_number_nodes(self):
    return len(self.nodes)

  def get_number_edges(self):
    return len(self.edges)
  
  def get_total_degree(self, node):
    return len(self.neighbour_reverse_set[node].\
               union(self.neighbour_set[node]))

In [3]:
# Refrence: https://snap.stanford.edu/data/ca-AstroPh.html

!gdown https://snap.stanford.edu/data/ca-AstroPh.txt.gz
!gzip -d ./ca-AstroPh.txt.gz

with open("/content/ca-AstroPh.txt", "r") as f:
  graph = Graph()

  lines = f.readlines()

  for line in lines[4:]:
    nodes = line.split("\t")
    graph.add_edge(Edge(source=int(nodes[0]), destination=int(nodes[1])))

Downloading...
From: https://snap.stanford.edu/data/ca-AstroPh.txt.gz
To: /content/ca-AstroPh.txt.gz
100% 1.45M/1.45M [00:00<00:00, 1.70MB/s]


In [4]:
graph.get_number_edges()

396160

In [5]:
graph.get_number_nodes()

18772

# Benefit Functions & Costs

In [6]:
import random
import numpy as np
def influence(graph:Graph, initial_nodes,
              probability=0.001, number_realization=1):

  effected_node_numbers = []
  for _ in range(number_realization):
    effected_nodes = set()
    influencers = initial_nodes

    while(len(influencers)>0):
      new_influencers = set()

      for influencer in influencers:
        effected_nodes.add(influencer)

      for influencer in influencers:
        neighbours = graph.neighbour_set[influencer]

        for neighbour in neighbours:
          if neighbour not in effected_nodes and random.random() < probability:
            new_influencers.add(neighbour)

      influencers = list(new_influencers)

    effected_node_numbers.append(len(effected_nodes))
  
  return np.average(effected_node_numbers)

In [7]:
def outbreak(graph:Graph, initial_nodes,
             probability=0.001, number_realization=1):
 
  sensor_nodes = initial_nodes
  
  for _ in range(number_realization):
    effected_nodes = set()
    initial_outbreak_nodes = random.sample(graph.nodes, 10)
    effected_node_numbers = []

    for outbreak_node in initial_outbreak_nodes:
      influencers = [outbreak_node]
      outbreak_detected = False

      while(len(influencers)>0 and not outbreak_detected):
        new_influencers = set()

        for influencer in influencers:
          effected_nodes.add(influencer)

        for influencer in influencers:
          neighbours = graph.neighbour_set[influencer]

          for neighbour in neighbours:
            if neighbour not in effected_nodes and random.random() < probability:
              new_influencers.add(neighbour)

              if neighbour in sensor_nodes:
                outbreak_detected = True

        influencers = list(new_influencers)

    effected_node_numbers.append(len(effected_nodes))
  
  not_effected_node_numbers = len(graph.nodes) - np.average(effected_node_numbers)

  return not_effected_node_numbers * 0.25

In [8]:
node_costs_q4 = dict()
for node in graph.nodes:
  node_costs_q4[node] = 1

In [9]:
node_costs_q5 = dict()
for node in graph.nodes:
  node_costs_q5[node] = graph.get_total_degree(node) * 0.8

# Hill Climbing & CELF

In [10]:
from copy import deepcopy
import math
def hill_climbing(graph:Graph,
                  budget:float,
                  benefit_function,
                  cost_normal:bool,
                  node_costs:dict,
                  probability=0.001,
                  number_realization=1):
  """
  Args:
    graph(Graph): The graph
    budget(float): The maximum treshold for costs of selected nodes.
    benefit_function(function): The benefit function (f)
    probability(float): The activation probability in the influence function. (See function influence)
    number_realization(int): The number of realization in the influence function. (See function influence)
    const_normal(bool): If this arg equals to true, the marginal gain will normalize with cost of each node.
    node_costs(dict): The dictionary that specify the selection cost for each node.

  Returns:
    list: The top nodes with maximum influence
  """
  
  top_nodes = list()
  remain_budget = budget

  while(True):
    best_marginal_benefit = -math.inf
    best_new_node = None

    for node in graph.nodes:
      if node in top_nodes or node_costs[node] > remain_budget: continue

      initial_nodes = deepcopy(top_nodes)
      initial_nodes.append(node)

      marginal_benefit = benefit_function(graph=graph,
                                                initial_nodes=initial_nodes,
                                                probability=probability,
                                                number_realization=number_realization)

      if cost_normal:
        marginal_benefit /= node_costs[node]

      if marginal_benefit > best_marginal_benefit:
        best_marginal_benefit = marginal_benefit
        best_new_node = node

    
    if best_new_node is not None:
      top_nodes.append(best_new_node)
      remain_budget -= node_costs[best_new_node]
      print(f"Top Nodes = {top_nodes}, Remain Budget = {remain_budget}")

    if remain_budget == 0 or best_new_node is None:
      break

  top_benefit = benefit_function(graph=graph,
                                 initial_nodes=top_nodes,
                                 probability=probability,
                                 number_realization=number_realization)

  return top_nodes, top_benefit

In [11]:
from copy import deepcopy
import math
import heapq

def lazy_hill_climbing(graph:Graph,
                       budget:float,
                       benefit_function,
                       cost_normal:bool,
                       node_costs:dict,
                       probability=0.001,
                       number_realization=1):
  """
  Args:
    graph(Graph): The graph
    budget(float): The maximum treshold for costs of selected nodes.
    benefit_function(function): The benefit function (f)
    probability(float): The activation probability in the influence function. (See function influence)
    number_realization(int): The number of realization in the influence function. (See function influence)
    const_normal(bool): If this arg equals to true, the marginal gain will normalize with cost of each node.
    node_costs(dict): The dictionary that specify the selection cost for each node.

  Returns:
    list: The top nodes with maximum influence
  """
  
  top_nodes = list()
  remain_budget = budget

  marginal_benefit_heap = list()
  heapq.heapify(marginal_benefit_heap)


  for node in graph.nodes:
    
      initial_benefit = benefit_function(graph=graph,
                                         initial_nodes=[node],
                                         probability=probability,
                                         number_realization=number_realization)
 
      if cost_normal:
        initial_benefit /= node_costs[node]

      heapq.heappush(marginal_benefit_heap, (-initial_benefit, (node, 0)))


  while(True):
    current_time = len(top_nodes)

    if len(marginal_benefit_heap)==0: break


    negative_node_benefit, (node, node_time) = heapq.heappop(marginal_benefit_heap)
    node_bebefit = -negative_node_benefit

    if node not in top_nodes and node_costs[node] <= remain_budget:
      if node_time == current_time:
        top_nodes.append(node)
        remain_budget -= node_costs[node]
        print(f"Top Nodes = {top_nodes}, Remain Budget = {remain_budget}")
        
        if remain_budget == 0:
          break

      else:

        initial_nodes = deepcopy(top_nodes)
        initial_nodes.append(node)

        marginal_benefit = benefit_function(graph=graph,
                                                  initial_nodes=initial_nodes,
                                                  probability=probability,
                                                  number_realization=number_realization)

        if cost_normal:
          marginal_benefit /= node_costs[node]

        negative_node_benefit = -marginal_benefit

        heapq.heappush(marginal_benefit_heap, (negative_node_benefit, (node, current_time)))


  top_benefit = benefit_function(graph=graph,
                                 initial_nodes=top_nodes,
                                 probability=probability,
                                 number_realization=number_realization)
  
  return top_nodes, top_benefit

In [23]:
def celf(graph:Graph,
        budget:float,
        benefit_function,
        node_costs:dict,
        probability=0.001,
        number_realization=1):

  """
  Args:
    graph(Graph): The graph
    budget(float): The maximum treshold for costs of selected nodes.
    benefit_function(function): The benefit function (f)
    probability(float): The activation probability in the influence function. (See function influence)
    number_realization(int): The number of realization in the influence function. (See function influence)
    node_costs(dict): The dictionary that specify the selection cost for each node.

  Returns:
    list: The top nodes with maximum influence
  """

  print("Ignore Cost Hill Climbing:")
  top_nodes_ignore_cost, benefit_ignore_cost = lazy_hill_climbing(graph=graph, budget=budget, benefit_function=benefit_function,
                                                                  cost_normal=False, node_costs=node_costs, probability=probability,
                                                                  number_realization=number_realization)
  
  print("Normal Cose Hill Climbing:")
  top_nodes_normal_cost, benefit_normal_cost = lazy_hill_climbing(graph=graph, budget=budget, benefit_function=benefit_function,
                                                                  cost_normal=True, node_costs=node_costs, probability=probability,
                                                                  number_realization=number_realization)
  
  if benefit_ignore_cost < benefit_normal_cost:
    return top_nodes_normal_cost, benefit_normal_cost
  else:
    return top_nodes_ignore_cost, benefit_ignore_cost

# Run Question 4

In [20]:
import random

sum_result = 0
for i in range(100):
  result = influence(graph=graph, initial_nodes=random.sample(graph.nodes, 10),
                    probability=0.01, number_realization=5)

  sum_result += result

print(sum_result/100)

19.770000000000003


In [21]:
import time

start = time.time()

nodes, benefit = hill_climbing(graph=graph,
                               budget=10,
                               benefit_function=influence,
                               probability=0.01,
                               number_realization=5, 
                               cost_normal=False, node_costs=node_costs_q4)

end = time.time()
print(f"Time = {end - start} seconds")
print(f"Nodes = {nodes}")
print(f"Benefit = {benefit}")

Top Nodes = [84424], Remain Budget = 9
Top Nodes = [84424, 131813], Remain Budget = 8
Top Nodes = [84424, 131813, 69236], Remain Budget = 7
Top Nodes = [84424, 131813, 69236, 14692], Remain Budget = 6
Top Nodes = [84424, 131813, 69236, 14692, 114160], Remain Budget = 5
Top Nodes = [84424, 131813, 69236, 14692, 114160, 501], Remain Budget = 4
Top Nodes = [84424, 131813, 69236, 14692, 114160, 501, 91812], Remain Budget = 3
Top Nodes = [84424, 131813, 69236, 14692, 114160, 501, 91812, 87227], Remain Budget = 2
Top Nodes = [84424, 131813, 69236, 14692, 114160, 501, 91812, 87227, 61196], Remain Budget = 1
Top Nodes = [84424, 131813, 69236, 14692, 114160, 501, 91812, 87227, 61196, 63243], Remain Budget = 0
Time = 451.0688045024872 seconds
Nodes = [84424, 131813, 69236, 14692, 114160, 501, 91812, 87227, 61196, 63243]
Benefit = 33.4


In [24]:
import time

start = time.time()

nodes, benefit = celf(graph=graph,
                      budget=10,
                      benefit_function=influence,
                      probability=0.01,
                      number_realization=5, 
                      node_costs=node_costs_q4)

end = time.time()
print(f"Time = {end - start} seconds")
print(f"Nodes = {nodes}")
print(f"Benefit = {benefit}")

Ignore Cost Hill Climbing:
Top Nodes = [53213], Remain Budget = 9
Top Nodes = [53213, 93504], Remain Budget = 8
Top Nodes = [53213, 93504, 111161], Remain Budget = 7
Top Nodes = [53213, 93504, 111161, 35348], Remain Budget = 6
Top Nodes = [53213, 93504, 111161, 35348, 37479], Remain Budget = 5
Top Nodes = [53213, 93504, 111161, 35348, 37479, 36833], Remain Budget = 4
Top Nodes = [53213, 93504, 111161, 35348, 37479, 36833, 97320], Remain Budget = 3
Top Nodes = [53213, 93504, 111161, 35348, 37479, 36833, 97320, 47968], Remain Budget = 2
Top Nodes = [53213, 93504, 111161, 35348, 37479, 36833, 97320, 47968, 58401], Remain Budget = 1
Top Nodes = [53213, 93504, 111161, 35348, 37479, 36833, 97320, 47968, 58401, 25850], Remain Budget = 0
Normal Cose Hill Climbing:
Top Nodes = [62821], Remain Budget = 9
Top Nodes = [62821, 114959], Remain Budget = 8
Top Nodes = [62821, 114959, 111161], Remain Budget = 7
Top Nodes = [62821, 114959, 111161, 72941], Remain Budget = 6
Top Nodes = [62821, 114959, 11

# Run Question 5

In [32]:
import random

sum_result = 0

for i in range(100):
  graph_nodes = list(graph.nodes)

  random.shuffle(graph_nodes)

  budget = 10

  selected_nodes = []
  for node in graph_nodes:
    if node_costs_q5[node] < budget:
      budget -= node_costs_q5[node]
      selected_nodes.append(node)

    
  result = outbreak(graph=graph, initial_nodes=random.sample(graph.nodes, 10),
                    probability=0.01, number_realization=5)

  sum_result += result

print(sum_result/100)

4687.605


In [26]:
import time

start = time.time()

nodes, benefit = hill_climbing(graph=graph,
                               budget=10,
                               benefit_function=outbreak,
                               probability=0.01,
                               number_realization=5, 
                               cost_normal=False, node_costs=node_costs_q5)

end = time.time()
print(f"Time = {end - start} seconds")
print(f"Nodes = {nodes}")
print(f"Benefit = {benefit}")

Top Nodes = [98326], Remain Budget = 6.8
Top Nodes = [98326, 65541], Remain Budget = 2.8
Top Nodes = [98326, 65541, 4], Remain Budget = 1.1999999999999997
Top Nodes = [98326, 65541, 4, 97], Remain Budget = 0.3999999999999997
Time = 95.18195247650146 seconds
Nodes = [98326, 65541, 4, 97]
Benefit = 4689.5


In [27]:
import time

start = time.time()

nodes, benefit = celf(graph=graph,
                      budget=10,
                      benefit_function=outbreak,
                      probability=0.01,
                      number_realization=5, 
                      node_costs=node_costs_q5)

end = time.time()
print(f"Time = {end - start} seconds")
print(f"Nodes = {nodes}")
print(f"Benefit = {benefit}")

Ignore Cost Hill Climbing:
Top Nodes = [35], Remain Budget = 6.0
Top Nodes = [35, 1102], Remain Budget = 1.1999999999999993
Top Nodes = [35, 1102, 2255], Remain Budget = 0.39999999999999925
Normal Cose Hill Climbing:
Top Nodes = [179], Remain Budget = 9.2
Top Nodes = [179, 3138], Remain Budget = 8.399999999999999
Top Nodes = [179, 3138, 3639], Remain Budget = 7.599999999999999
Top Nodes = [179, 3138, 3639, 4035], Remain Budget = 6.799999999999999
Top Nodes = [179, 3138, 3639, 4035, 5414], Remain Budget = 5.999999999999999
Top Nodes = [179, 3138, 3639, 4035, 5414, 11039], Remain Budget = 5.199999999999999
Top Nodes = [179, 3138, 3639, 4035, 5414, 11039, 11153], Remain Budget = 4.3999999999999995
Top Nodes = [179, 3138, 3639, 4035, 5414, 11039, 11153, 15353], Remain Budget = 3.5999999999999996
Top Nodes = [179, 3138, 3639, 4035, 5414, 11039, 11153, 15353, 21370], Remain Budget = 2.8
Top Nodes = [179, 3138, 3639, 4035, 5414, 11039, 11153, 15353, 21370, 21383], Remain Budget = 1.9999999999